<a href="https://colab.research.google.com/github/JoosepAlbre/IDS_Project/blob/main/Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [262]:
import pandas as pd

url = 'https://raw.githubusercontent.com/JoosepAlbre/IDS_Project/main/model.csv'
data = pd.read_csv(url)

In [263]:
#Original length
length = len(data)

#Drop column index 0 and row with 'unknown' sex
data = data.drop(data.columns[0], axis=1)
data = data.drop(data[data.Sex == "Unknown"].index)

#DropNa
data = data.dropna()

data = data[['Age', 'Sex', 'Race', 'Fentanyl', 'Heroin', 'Benzodiazepine', 'AnyOpioid', 'Cocaine', 'DeathCity']]
print(data.columns)

# Remove multiple race intenticators
index = 0
for i in data['Race']:
  if "," in str(i):
    race = i.split(',')[0]
    data.iloc[index, data.columns.get_loc('Race')] = race
  index += 1

#Remove instances with rare race occurances
data = data.loc[~( (data['Race'] != 'White') & (data['Race'] != 'Black') & (data['Race'] != 'Hispanic') ),:]

data["COD_Drug"] = 0
columns = data.columns

#Find COD_Drug
for i in range(length):
  for j in range (3, 8):
    element = data[columns[j]].get(i)

    if element == 1:
      if data['COD_Drug'][i] == 0:
        data['COD_Drug'][i] = str(j)
        break


data['Race'].value_counts()
data['COD_Drug'].value_counts()

#print(data['COD_Drug'].value_counts())
data = data.drop(data.columns[[3,4,5,6,7]], axis=1)

Index(['Age', 'Sex', 'Race', 'Fentanyl', 'Heroin', 'Benzodiazepine',
       'AnyOpioid', 'Cocaine', 'DeathCity'],
      dtype='object')


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [264]:
#Creating a RFC model
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC


#Change categorical values to binary
data_dum = pd.get_dummies(data, columns=["Race", 'DeathCity'])


#Divide the dataset to test and train set
X_train, X_test, y_train, y_test = train_test_split(data_dum.loc[:, data_dum.columns != 'COD_Drug'], data_dum['COD_Drug'], test_size=0.3)


###### (Semi-Failed)  Balanced dataset
least_ODs = (data_dum['COD_Drug'] == 6).sum() #Number of lowest death causing substance ODs
fentanyl = data_dum[data['COD_Drug'] == 3].sample(least_ODs)
heroin = data_dum[data['COD_Drug'] == 4].sample(least_ODs)
benzo = data_dum[data['COD_Drug'] == 5].sample(least_ODs)
anyOpioid = data_dum[data['COD_Drug'] == 6]
cocaine = data_dum[data['COD_Drug'] == 7].sample(least_ODs)
other = data_dum[data['COD_Drug'] == 0].sample(least_ODs)

train_bal = pd.concat([fentanyl, heroin, benzo, anyOpioid, cocaine, other])

X_train_bal = train_bal.loc[:, train_bal.columns != 'COD_Drug']
y_train_bal = train_bal['COD_Drug']

rf_bal = RandomForestClassifier(n_estimators=100, max_depth=4, random_state=0).fit(X_train_bal, y_train_bal)
cod_pred_bal = rf_bal.predict(X_test)
acc_bal = accuracy_score(y_test, cod_pred_bal)
print("(Should have been a) Balanced : " + str(acc_bal))
######

# Train the model
rf = RandomForestClassifier(n_estimators=1000, max_depth=5).fit(X_train, y_train)
knn = KNeighborsClassifier(n_neighbors = 99).fit(X_train, y_train)
svc = SVC(kernel = 'poly', degree = 2, gamma = 'auto').fit(X_train, y_train)

# Predict
rf_pred = rf.predict(X_test)
knn_pred = knn.predict(X_test)
svc_pred = svc.predict(X_test)

# Find the accuracy
acc_rf = accuracy_score(y_test, rf_pred)
acc_knn = accuracy_score(y_test, knn_pred)
acc_svc = accuracy_score(y_test, svm_pred)

print("ACCURACY: \n rf: " + str(acc_rf) + "\n knn: " + str(acc_knn)  + "\n svm: " + str(acc_svc))

accuracys = [acc_bal, acc_rf, acc_knn, acc_svc]

acc = pd.DataFrame(columns=['Name', 'Accuracy'])
acc = acc.append({'Name': 'Random Forest Classifier', 'Accuracy': acc_rf},ignore_index=True)
acc = acc.append({'Name': 'Random Forest on balanced data', 'Accuracy': acc_bal},ignore_index=True)
acc = acc.append({'Name': 'K-Nearest Neighbours', 'Accuracy': acc_knn},ignore_index=True)
acc = acc.append({'Name': 'C-Support Vector Classification', 'Accuracy': acc_svc},ignore_index=True)

print(acc)


(Should have been a) Balanced : 0.3136212624584718
ACCURACY: 
 rf: 0.45382059800664454
 knn: 0.4485049833887043
 svm: 0.45382059800664454
                              Name  Accuracy
0         Random Forest Classifier  0.453821
1   Random Forest on balanced data  0.313621
2             K-Nearest Neighbours  0.448505
3  C-Support Vector Classification  0.453821
